In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from skimage import measure
from shapely.geometry.polygon import Polygon
import folium

In [2]:
path_data = "/glade/p/cisl/aiml/jtti_tornado/wofs_predictions/2019/"
path_save = "/glade/p/cisl/aiml/jtti_tornado/unet_donuts/2019/"

var_intensities = {'UH': [25, 50, 75, 100, 125, 150],
                   'ZH_1km': np.arange(10, 85, 5),
                   'predicted_tor': np.arange(.10, .70, .10)}
var_save_str = {'UH': 'uh',
                'ZH_1km': 'zh',
                'predicted_tor': 'tor'}

In [3]:
for date in next(os.walk(path_data))[1]:
    path_date = os.path.join(path_data, date)
    os.makedirs(path_date, exist_ok=True)
    directories = [f[0] for f in os.walk(path_date) if f[0].endswith('ENS_MEM', 68, 75)]
    for directory in directories[:1]:
        ds = xr.open_dataset(os.path.join(directory, "wofs_stitched_light.nc"))
        identifier = directory.split("/")[-1]
        lons = ds['lon'] - 360
        lats = ds['lat']
        for var in ['UH', 'ZH_1km', 'predicted_tor']:
            for time in ds["time"].values:
                df = pd.DataFrame()
                var_grid = ds.sel(time=time).to_dataframe()[var].unstack('lat').values
        break
                # for intensity in var_intensities[var][:1]:
                #     contours = measure.find_contours(var_grid, intensity)
                #     for i, contour in enumerate(contours):
                #         if contour.shape[0] < 3:
                #             continue
                #         else:
                #             contour = np.round(contour).astype(int)
                #             lons_contour = lons[contour[:, 0:1].flatten()].values.reshape(-1,1)
                #             lats_contour = lats[contour[:, 1:2].flatten()].values.reshape(-1,1)
                #             contour = np.hstack((lons_contour, lats_contour))
                #             contour_dict = {"intensity": intensity, "time":time, "geometry": Polygon(contour)}
                #             df = df.append(contour_dict, ignore_index=True)
                # if df.shape[0] > 0:
                #     gdf = gpd.GeoDataFrame(df, geometry="geometry").set_crs("EPSG:4326")
                #     time = pd.to_datetime(str(time)).strftime("%Y%m%d_%H%M%S")
                #     file = f'{var_save_str[var]}_{identifier}_{time}.geojson'
                #     gdf.to_file(os.path.join(path_save, date, file), driver='GeoJSON')

In [2]:
var_grid

NameError: name 'var_grid' is not defined

In [5]:
var_grid.shape

(427, 380)

In [15]:
print(ds[var].sel(time=time).shape)
var_grid = ds[var].sel(time=time)

(380, 427)


https://stackoverflow.com/questions/48770822/how-to-make-holes-in-a-polygon-in-shapely-python-having-polygons

In [18]:
contours = measure.find_contours(var_grid.values, var_intensities[var][0])
# contour_0 = np.round(contour_0).astype(int)
# lons_contour_0 = lons[contour_0[:, 0:1].flatten()].values.reshape(-1,1)
# lats_contour_0 = lats[contour_0[:, 1:2].flatten()].values.reshape(-1,1)
# contour_0 = np.hstack((lons_contour_0, lats_contour_0))

# contour_1 = measure.find_contours(var_grid, var_intensities[var][1])
# contour_1 = np.round(contour_1).astype(int)
# lons_contour_1 = lons[contour_1[:, 0:1].flatten()].values.reshape(-1,1)
# lats_contour_1 = lats[contour_1[:, 1:2].flatten()].values.reshape(-1,1)
# contour_1 = np.hstack((lons_contour_1, lats_contour_1))

In [21]:
print(len(contours))
for contour in contours:
    print(contour.shape)

26
(7, 2)
(7, 2)
(23, 2)
(27, 2)
(19, 2)
(63, 2)
(29, 2)
(7, 2)
(33, 2)
(151, 2)
(5, 2)
(27, 2)
(5, 2)
(75, 2)
(55, 2)
(41, 2)
(11, 2)
(11, 2)
(13, 2)
(69, 2)
(15, 2)
(27, 2)
(25, 2)
(23, 2)
(13, 2)
(29, 2)


In [ ]:
style1 = {'fillColor': '#4E79A7', 'color': '#4E79A7'}
style2 = {'fillColor': '#A0CBE8', 'color': '#A0CBE8'}
style3 = {'fillColor': '#F28E2B', 'color': '#F28E2B'}
style4 = {'fillColor': '#FFBE7D', 'color': '#FFBE7D'}
style5 = {'fillColor': '#59A14F', 'color': '#59A14F'}
style6 = {'fillColor': '#8CD18D', 'color': '#8CD18D'}
style7 = {'fillColor': '#B6992D', 'color': '#B6992D'}
style8 = {'fillColor': '#F1CE63', 'color': '#F1CE63'}
style9 = {'fillColor': '#499894', 'color': '#499894'}
style10 = {'fillColor': '#86BCB6', 'color': '#86BCB6'}
style11 = {'fillColor': '#E15759', 'color': '#E15759'}
style12 = {'fillColor': '#FF9D9A', 'color': '#FF9D9A'}
style13 = {'fillColor': '#79706E', 'color': '#79706E'}
style14 = {'fillColor': '#BAB0AC', 'color': '#BAB0AC'}
style15 = {'fillColor': '#D37295', 'color': '#D37295'}
style16 = {'fillColor': '#FABFD2', 'color': '#FABFD2'}
style17 = {'fillColor': '#B07AA1', 'color': '#B07AA1'}
style18 = {'fillColor': '#D4A6C8', 'color': '#D4A6C8'}
style19 = {'fillColor': '#9D7660', 'color': '#9D7660'}
style20 = {'fillColor': '#D7B5A6', 'color': '#D7B5A6'}

In [ ]:
m = folium.Map(location=[40, -90], zoom_start=5, tiles='CartoDB positron')
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==25]['geometry'], style_function=lambda x: style1).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==50]['geometry'], style_function=lambda x: style2).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==75]['geometry'], style_function=lambda x: style3).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==100]['geometry'], style_function=lambda x: style4).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==125]['geometry'], style_function=lambda x: style5).add_to(m)
m

In [ ]:
style1 = {'fillColor': '#003f5c', 'color': '#003f5c'}
m = folium.Map(location=[40, -90], zoom_start=5, tiles='CartoDB positron')
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==10]['geometry'], style_function=lambda x: style1).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==15]['geometry'], style_function=lambda x: style2).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==20]['geometry'], style_function=lambda x: style3).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==25]['geometry'], style_function=lambda x: style4).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==30]['geometry'], style_function=lambda x: style5).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==35]['geometry'], style_function=lambda x: style6).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==40]['geometry'], style_function=lambda x: style7).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==45]['geometry'], style_function=lambda x: style8).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==50]['geometry'], style_function=lambda x: style9).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==55]['geometry'], style_function=lambda x: style10).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==60]['geometry'], style_function=lambda x: style11).add_to(m)
m

In [ ]:
style1 = {'fillColor': '#003f5c', 'color': '#003f5c'}
m = folium.Map(location=[40, -90], zoom_start=5, tiles='CartoDB positron')
folium.GeoJson(data=gdf_st[gdf_st['intensity'] == 0.1]['geometry'], style_function=lambda x: style1).add_to(m)
folium.GeoJson(data=gdf_st[gdf_st['intensity'] == 0.2]['geometry'], style_function=lambda x: style2).add_to(m)
m

In [24]:
(33426 - 24648) * 30

263340

In [26]:
22230818 + 263340 + 1996*250

22993158

In [28]:
(365/12 * 24 * 17) / 60

206.83333333333334

In [29]:
import time
import datetime

start = time.time()

0:00:00.000010


In [30]:
end = time.time()
delta = end - start
print(datetime.timedelta(seconds=delta))

0:00:11.910995
